# Landmark analysis

Investigate the role socio-economic factors have in the development of areas around Eurpoean Landmarks, by looking at their effects on landmark clustering.

Quickly assess the degree to which landmark popularity is independent of socio-economic factors. If determined to be independent, then this in turn can be used to maximise investment in areas in high return (high price/popularity) and low cost (low cost of living indicators) 

Cluster initially only by the price and rating of venues in close proximity to the Landmark. Add in socio-economic factors and cluster again, visually comparing the results.

## Load the landmarks

First load the local Landmarks csv file that we've created and view

In [1]:
import pandas as pd
df_landmarks = pd.read_csv('Landmarks.csv')
df_landmarks.head()

,Landmark,City,Country,Latitude,Longitude
0,Colosseum,Rome,Italy,41.8902,12.4922
1,Eiffel Tower,Paris,France,48.8584,2.2945
2,Sagrada Familia,Barcelona,Spain,41.4036,2.1744
3,The Church of Hallgrimur,Reykjavik,Iceland,64.1417,-21.9266
4,Leaning Tower of Pisa,Pisa,Italy,43.7230,10.3966


## Now load information regarding the landmarks from FourSquare

We get the venues within 500m of each landmark, then we look at the price and popularity of these venues. We use a number of methods to help us achieve this, so lets declare them first

### Method to retrieve Foursquare feeds via a local cache
All FourSquare APIs will go through this. We are not time sensitive, so the retrieved API requests can be cached for future use to prevent us going over the limit of FourSquare API throttling. This is especially important as -

1. We want to make a lot of API calls (to get averages)
2. We need to be able to repeat calls when testing

To make matters worse, the price and rating information requires a 'premium' call (restricted to 50 API calls per day for a sandbox account), so we may also need to restrict the number of Landmarks being considered in this study.

In [2]:
CLIENT_ID = 'LKJ2HMDVO3MXO2MPSE5QO5R2HPLONX0YHRT4J23I3QCMKCXW' # your Foursquare ID
CLIENT_SECRET = 'VVQRKXFSIUI4TD15OFEDPI3VKJX5OFBGKWKBMUHG245GNWT4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# Lets keep some statistics on this
basic_stats = {"from_cache":0, "from_api":0, "errors":0}
premium_stats = {"from_cache":0, "from_api":0, "errors":0}

In [3]:
import requests
import json
from pathlib import Path
import numpy as np
from pandas.io.json import json_normalize

def get_venues_close(lat, long, section):
    vurl = 'https://api.foursquare.com/v2/venues/explore?&ll={},{}&section={}radius={}&limit={}&'.format(
        lat, 
        long, 
        section,
        500, 
        100) # yes the '&' is ugly but it saves us a four line if !
    vpath = Path('ResponseCache/Explore.{}.{}.{}.json'.format(lat,long,section))
    return get_cached_call(vpath, vurl, basic_stats)

def get_venue_details(venueid):
    vurl = 'https://api.foursquare.com/v2/venues/{}?'.format(venueid) # yes the '?' is ugly but it saves us a four line if !
    vpath = Path('ResponseCache/Venue.{}.json'.format(venueid))
    return get_cached_call(vpath, vurl, premium_stats)
    
def get_cached_call(json_path, api_url, stats):
    if json_path.exists():
        stats["from_cache"]=stats["from_cache"]+1
        f = open(json_path, "r")
        results = json.load(f);
    else:
        stats["from_api"]=stats["from_api"]+1
        url = api_url+'client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
        results = requests.get(url).json()
        if results['meta']['code']==200:
            f = open(json_path, "w")
            json.dump(results,f)
        else:
            stats["errors"]=stats["errors"]+1
    
    return results

### Method to determine if a row retrieved represents a restaurant
This method is a simple check for the presence of the Restaurant Category

In [4]:
# function that validates the presence of the category 'Restaurant'
def is_restaurant(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    size = len(categories_list)
    if size == 0:
        return False

    hasCategory = False
    for category in categories_list:
        if 'Restaurant' in categories_list[0]['name']: # == not sufficient as ethnic restaurant include the type
            hasCategory = True
    
    return hasCategory

### Method to extract only rows corresponding to restaurants from an /explore query
We create a method that will filter the rows found to only those that are Restaurants as we want to compare restaurant prices and ratings as the primary clustering criteria, and food includes many more categories. Also, as we will be doing individual queries on all, we will only need the id column (but include name for clarity)

In [5]:
def get_restaurant_ids(results):
    venues = results['response']['groups'][0]['items']
    df_venues = json_normalize(venues) # Gives a huge number of columns

    # Now pick out the columns we're interested in and filter to restaurants ('food' category alone doesn't do this)
    filtered_columns = ['venue.name', 'venue.id', 'venue.categories']
    df_venues = df_venues.loc[:, filtered_columns]
    return df_venues[df_venues.apply(is_restaurant, axis=1)]

### Method to get the price and rating for any given venue
First we define the FourSquare access details in an individual cell (we won't need to repeat any of this when testing). After that we create two methods for use later.
- Given a venue id, the price (unavailable from the FourSquare API) and rating are returned using the venue API
- Given a Series of venue ids, the average of the price and ratings are returned

In [6]:
def get_venue_price_and_rating(venueid):
    results = get_venue_details(venueid)
    try :
        return results['response']['venue']['price']['tier'], results['response']['venue']['rating']
    except:
        return None, None # ignore poorly formatted results, and ensure mean generation will skip

def get_average_price_and_rating(s_venueids):
    data = [[0 for x in range(2)] for y in range(s_venueids.size)]
    for idx, venueid in enumerate(s_venueids):
        price,rating = get_venue_price_and_rating(venueid)
        data[idx][0] = price
        data[idx][1] = rating
    df = pd.DataFrame(data, columns=['Price','Rating'])
    return df['Price'].mean(skipna=True), df['Rating'].mean(skipna=True)

In [7]:
#TEST CODE based on the Coloseum  - DO NOT RUN !!!!!!!!
results = get_venues_close('41.8902','12.4922','food')
restaurants = get_restaurant_ids(results)
rids = restaurants['venue.id']
results = get_venue_details(rids[6]);

avgp, avgr = get_average_price_and_rating(rids)
avgp, avgr

(2.9473684210526314, 8.147368421052633)

### Now iterate through the landmarks
For each landmark we get the restaurants and then find the rating for each

In [8]:
section='food'
prices = []
ratings = []

df_landmark_details = df_landmarks

for index, row in df_landmark_details.iterrows():
    print(row[0]) # as this takes a while to run - need feedback
    results = get_venues_close(row[3],row[4],'food')
    df_restaurants = get_restaurant_ids(results)
    
    df_restaurants = df_restaurants.head(20) # populate the cache at least a little for all landmarks - we'll increase this later
    
    rids = df_restaurants['venue.id']
    avgp, avgr = get_average_price_and_rating(rids)
    prices.append(avgp)
    ratings.append(avgr)

df_landmark_details = df_landmark_details.assign(Prices=pd.Series(prices))
df_landmark_details = df_landmark_details.assign(Ratings=pd.Series(ratings))

Colosseum
Eiffel Tower
Sagrada Familia
The Church of Hallgrimur
Leaning Tower of Pisa
Brandenburg Gate
Ancient City Walls of Dubrovnik
Acropolis
Duomo
Arc de Triomphe
Alhambra
Big Ben
Buckingham Palace
Trevi Fountain
Pantheon
Charles Bridge
Schonbrunn Palace 
Red Square
Church of Our Savior on Spilled Blood
Belem Tower
La Grand-Place
Dublin Castle
Anne Frank House
Guggenheim Museum


### Print out statistics of cache usage
A quick checkpoint to see how many calls are cached and still in error

In [9]:
print(basic_stats)
print(premium_stats)

{'from_cache': 25, 'from_api': 0, 'errors': 0}
{'from_cache': 470, 'from_api': 0, 'errors': 0}


## Load the City Statistics

Numbeo offer free visual data, but charge for their API, so data has instead been copied and downloaded to files locally. Here we look at two data sets that are of particular interest and normalize the city names (removing the country) in order to match the City names used above. The two Files are -
- `CitySalaries.csv` which offers the monthly salary per city
- `CityCostOfLiving.csv` this offers a number of statistics such as restaurant prices and rent


In [10]:
df_salaries = pd.read_csv('CitySalaries.csv')
split = df_salaries['City'].str.split(',')
df_salaries.drop(['City'],axis=1)
df_salaries['City']=split.str[0]
df_salaries['Country']=split.str[1]

# and merge in salary
df_landmarks_with_salary = pd.merge(df_landmark_details, df_salaries[['City','Salary']], left_on='City',right_on='City', how='left')
df_landmarks_with_salary.dropna(axis=0, inplace=True)
df_landmarks_with_salary.head(2)

,Landmark,City,Country,Latitude,Longitude,Prices,Ratings,Salary
0,Colosseum,Rome,Italy,41.8902,12.4922,2.947368,8.147368,1638.22
1,Eiffel Tower,Paris,France,48.8584,2.2945,2.900000,8.690000,2784.38


In [11]:
df_living = pd.read_csv('CityCostOfLiving.csv')
split = df_living['City'].str.split(',')
df_living.drop(['City'],axis=1)
df_living['City']=split.str[0]
df_living['Country']=split.str[1]

df_complete = pd.merge(df_landmarks_with_salary, df_living[['City','Cost of Living Index','Rent Index', 'Groceries Index']], left_on='City',right_on='City', how='left')
df_complete.dropna(axis=0, inplace=True)
df_complete.head(2)

,Landmark,City,Country,Latitude,Longitude,Prices,Ratings,Salary,Cost of Living Index,Rent Index,Groceries Index
0,Colosseum,Rome,Italy,41.8902,12.4922,2.947368,8.147368,1638.22,78.80,40.24,68.34
1,Eiffel Tower,Paris,France,48.8584,2.2945,2.900000,8.690000,2784.38,92.87,50.30,87.29


## Investigate variance and correlation of data

### Mean, Standard Deviation and Variance

Before looking at the clustering we should be aware of what the variance (and therefore default 'bias') will be with all of the columns that we could potentially choose to use for clustering. KMeans does some normalization, but doesn't fully address differences in Variance.


In [12]:
df_complete[['Prices','Ratings','Cost of Living Index','Rent Index','Groceries Index']].std()

Prices                   0.349621
Ratings                  0.208975
Cost of Living Index    17.205251
Rent Index              17.492027
Groceries Index         17.741422
dtype: float64

However, we really need to see the standard deviation as a fraction of the mean (also known as the coefficient of variation) to get an idea for what kind of normalization will be required.

In [13]:
stat_cols = df_complete[['Prices','Ratings','Cost of Living Index','Rent Index','Groceries Index']]
pd.DataFrame({'StdDev':stat_cols.std(),'Mean':stat_cols.mean(), 'Variation':stat_cols.std()/stat_cols.mean()})

,StdDev,Mean,Variation
Prices,0.349621,2.420003,0.144471
Ratings,0.208975,8.397633,0.024885
Cost of Living Index,17.205251,78.905500,0.218049
Rent Index,17.492027,41.974000,0.416735
Groceries Index,17.741422,67.490500,0.262873


### Correlation and feature selection/removal

Rather than doing complex feature removal we should simply try to look at correlation of the city features and see if there are any that could be considered superfluous, in case this is sufficient.

In [14]:
#print(df_complete['Cost of Living Index'].corr(df_complete['Rent Index']))
df_complete[['Cost of Living Index','Rent Index','Groceries Index']].corr()

,Cost of Living Index,Rent Index,Groceries Index
Cost of Living Index,1.000000,0.692002,0.960791
Rent Index,0.692002,1.000000,0.526091
Groceries Index,0.960791,0.526091,1.000000


## Now cluster and display

We will run a clustering algorithm on the landmarks based on their price and rating averages. This will identify a range of cities that share price/rating averages. Cities that cluster differently when socio-economic indicators are taken into account will present an immediate visual indicator of potential development opportunities.

### First some setup

Colours and some time consuming imports

In [15]:
!conda install -c conda-forge folium=0.5.0 --yes

import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage and folium (assumes already installed)
from sklearn.cluster import KMeans
import folium # map rendering library

# set color scheme for the clusters
numclusters = 4
x = np.arange(numclusters)
ys = [i+x+(i*x)**2 for i in range(numclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    altair-2.2.2               |        py36_1001         494 KB  conda-forge
    ------------------------------------------------------------
                         

### Clustering 1 - with just price and ratings

Extract the prices and ratings and cluster 

In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Scale
df_landmark_stats1 = df_complete[['Prices','Ratings']]
scaler = StandardScaler()
scaler.fit(df_landmark_stats1)
df_scaled1 = scaler.transform(df_landmark_stats1)

#Cluster
kmeans1 = KMeans(n_clusters=numclusters, random_state=0).fit(df_scaled1)
df_clustered1 = df_complete.assign(Cluster = pd.Series(kmeans1.labels_))
df_clustered1.head()


,Landmark,City,Country,Latitude,Longitude,Prices,Ratings,Salary,Cost of Living Index,Rent Index,Groceries Index,Cluster
0,Colosseum,Rome,Italy,41.8902,12.4922,2.947368,8.147368,1638.22,78.80,40.24,68.34,1
1,Eiffel Tower,Paris,France,48.8584,2.2945,2.900000,8.690000,2784.38,92.87,50.30,87.29,2
2,Sagrada Familia,Barcelona,Spain,41.4036,2.1744,2.000000,8.189474,1665.35,69.75,33.71,59.29,3
3,The Church of Hallgrimur,Reykjavik,Iceland,64.1417,-21.9266,2.200000,8.395000,3226.82,123.78,57.25,118.15,0
4,Brandenburg Gate,Berlin,Germany,52.5163,13.3777,2.250000,8.358333,2432.13,74.32,33.54,60.74,0


And now display graphically

In [17]:
# create map of Europe using latitude and longitude values (these needed manual tweaking for best display)
EUROPE_LAT = 52
EUROPE_LONG = 15
map_europe = folium.Map(location=[EUROPE_LAT, EUROPE_LONG], zoom_start=4)

# add markers to the map
for index, row in df_clustered1.iterrows():
    cluster = row['Cluster']
    label = '{}, {}'.format(row['Landmark'], row['City'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_europe)  

    
map_europe

Now lets do a quick analysis of what the clusters mean by finding some averages

In [18]:
df_clusters = df_clustered1[['Prices','Ratings','Cluster']].groupby(['Cluster']).mean()
df_clusters.assign(Color = ['Purple','Blue','Yellow','Red'])

,Prices,Ratings,Color
Cluster,,,
0,2.224966,8.401798,Purple
1,2.911842,8.313092,Blue
2,2.588248,8.701784,Yellow
3,2.125000,8.092325,Red


## Clustering 2 with socio economic factors

Take the columns from earlier and update see how the clustering differs

In [19]:
from sklearn.cluster import KMeans

#df_landmark_stats2 = df_complete[['Cost of Living Index','Rent Index', 'Groceries Index']]
df_landmark_stats2 = df_complete[['Prices','Ratings','Rent Index', 'Groceries Index']]

scaler = StandardScaler()
scaler.fit(df_landmark_stats2)
df_normalised = scaler.transform(df_landmark_stats2)

kmeans1 = KMeans(n_clusters=numclusters, random_state=0).fit(df_normalised)
df_clustered2 = df_complete.assign(Cluster = pd.Series(kmeans1.labels_))
df_clustered2.head()

,Landmark,City,Country,Latitude,Longitude,Prices,Ratings,Salary,Cost of Living Index,Rent Index,Groceries Index,Cluster
0,Colosseum,Rome,Italy,41.8902,12.4922,2.947368,8.147368,1638.22,78.80,40.24,68.34,3
1,Eiffel Tower,Paris,France,48.8584,2.2945,2.900000,8.690000,2784.38,92.87,50.30,87.29,0
2,Sagrada Familia,Barcelona,Spain,41.4036,2.1744,2.000000,8.189474,1665.35,69.75,33.71,59.29,1
3,The Church of Hallgrimur,Reykjavik,Iceland,64.1417,-21.9266,2.200000,8.395000,3226.82,123.78,57.25,118.15,2
4,Brandenburg Gate,Berlin,Germany,52.5163,13.3777,2.250000,8.358333,2432.13,74.32,33.54,60.74,1


Set up a manually determined rainbow

In [20]:
rainbow2 = [rainbow[2],rainbow[3],rainbow[0],rainbow[1]]
rainbow2

['#d4dd80', '#ff0000', '#8000ff', '#2adddd']

and now display graphically

In [21]:
# create map of Europe using latitude and longitude values (these needed manual tweaking for best display)
map_europe = folium.Map(location=[EUROPE_LAT, EUROPE_LONG], zoom_start=4)

# add markers to the map
for index, row in df_clustered2.iterrows():
    cluster = row['Cluster']
    label = '{}, {}'.format(row['Landmark'], row['City'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color=rainbow2[cluster],
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_europe)  

    
map_europe

In [22]:
df_clusters = df_clustered2[['Cost of Living Index','Rent Index','Prices','Ratings','Cluster']].groupby(['Cluster']).mean()
df_clusters.assign(Color = ['Yellow','Purple','Red','Blue'])

,Cost of Living Index,Rent Index,Prices,Ratings,Color
Cluster,,,,,
0,91.476667,59.090000,2.707407,8.699815,Yellow
1,60.937143,26.372857,2.225404,8.381710,Purple
2,92.303333,54.178333,2.175439,8.321480,Red
3,80.825000,38.132500,2.911842,8.313092,Blue
